In [1]:
pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.6 MB/s eta 0:00:00:00:0100:01
ERROR: Operation cancelled by user


In [ ]:
!pip install -U bitsandbytes

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import threading
import torch
import bitsandbytes
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load the model configuration and explicitly disable quantization settings
# config = AutoConfig.from_pretrained("Tann-dev/sex-chat-dirty-girlfriend")
# quantization_config = None
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
# config.quantization_config = None  # disable quantization

# Load tokenizer and model without quantization kwargs

model = AutoModelForCausalLM.from_pretrained(
    "Tann-dev/sex-chat-dirty-girlfriend",
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained("Tann-dev/sex-chat-dirty-girlfriend")


def generate_response(prompt):
    """
    Encodes the prompt, generates a response, and decodes the output.
    Adjust generation parameters as needed.
    """
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate a response using sampling for diversity.
    output_ids = model.generate(
        input_ids,
        max_length=150,          # Adjust as needed
        do_sample=True,          # Enable sampling for more varied outputs
        top_k=50,                # Consider the top 50 tokens at each step
        top_p=0.95,              # Use nucleus sampling with p=0.95
        temperature=0.7,         # Adjust temperature for randomness
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

def update_chat_display(sender, message):
    """
    Inserts the message into the chat window.
    """
    chat_display.config(state='normal')
    chat_display.insert(tk.END, f"{sender}: {message}\n")
    chat_display.config(state='disabled')
    chat_display.yview(tk.END)

def process_user_message(message):
    """
    Prepares the prompt and gets the model's response, then updates the chat display.
    """
    # You can adjust the prompt format based on your model's requirements.
    prompt = f"User: {message}\nBot:"
    try:
        bot_reply = generate_response(prompt)
        update_chat_display("Bot", bot_reply)
    except Exception as e:
        update_chat_display("Bot", f"[Error]: {e}")

def send_message(event=None):
    """
    Retrieves the user's message, updates the chat display,
    and starts a thread to get the bot's response.
    """
    message = user_input.get().strip()
    if message:
        update_chat_display("User", message)
        user_input.delete(0, tk.END)
        # Use a separate thread so the GUI doesn't freeze during generation.
        threading.Thread(target=process_user_message, args=(message,), daemon=True).start()

# Create the main application window
root = tk.Tk()
root.title("Chat Window")

# Create a scrolled text widget to display the conversation
chat_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', width=80, height=20)
chat_display.pack(padx=10, pady=10)

# Create an entry widget for user input
user_input = tk.Entry(root, width=80)
user_input.pack(padx=10, pady=(0, 10))
user_input.bind("<Return>", send_message)

# Create a Send button
send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack(pady=(0, 10))

# Start the Tkinter event loop
root.mainloop()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

TclError: no display name and no $DISPLAY environment variable

In [ ]:
import threading
import torch
import bitsandbytes  # ensure bitsandbytes is installed if required
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import ipywidgets as widgets
from IPython.display import display

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up quantization config (if you require 4-bit quantization; otherwise, remove it)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "Tann-dev/sex-chat-dirty-girlfriend",
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained("Tann-dev/sex-chat-dirty-girlfriend")

# Move model to device
model.to(device)

def generate_response(prompt):
    """
    Encodes the prompt, generates a response, and decodes the output.
    Adjust generation parameters as needed.
    """
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(
        input_ids,
        max_length=150,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# --- Build the Colab chat interface using ipywidgets with a HTML widget for chat log ---

# Global variable to store the chat log as HTML
chat_log = ""

# HTML widget for displaying the chat
chat_display = widgets.HTML(
    value=chat_log,
    layout=widgets.Layout(border='1px solid black', height='300px', overflow_y='scroll', width='100%')
)

# Text input widget for user message
user_input = widgets.Text(placeholder="Type your message here...", layout=widgets.Layout(width='80%'))

# Send button widget
send_button = widgets.Button(description="Send", layout=widgets.Layout(width='18%'))

def append_message(sender, message):
    """Append a message to the chat log and update the HTML widget."""
    global chat_log
    # Append new message using simple HTML formatting
    chat_log += f"<p><strong>{sender}:</strong> {message}</p>"
    chat_display.value = chat_log

def process_user_message(message):
    """Generate the bot response in a separate thread and update chat display."""
    prompt = f"User: {message}\nBot:"
    try:
        bot_reply = generate_response(prompt)
        append_message("Bot", bot_reply)
    except Exception as e:
        append_message("Bot", f"[Error]: {e}")

def on_send_clicked(b):
    message = user_input.value.strip()
    if message:
        append_message("User", message)
        user_input.value = ""  # Clear the input field
        threading.Thread(target=process_user_message, args=(message,), daemon=True).start()

# Bind events: click on send button or press Enter in text input
send_button.on_click(on_send_clicked)
user_input.on_submit(lambda _: on_send_clicked(None))

# Display the chat interface
display(chat_display, widgets.HBox([user_input, send_button]))


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


HTML(value='', layout=Layout(border='1px solid black', height='300px', overflow_y='scroll', width='100%'))